# OOP trong Python — Notebook (Chi tiết, Định dạng sạch)

**Mục tiêu:** giải thích 4 tính chất OOP (Đóng gói, Trừu tượng, Kế thừa, Đa hình).

## Tổng quan ngắn

OOP (Object-Oriented Programming) tổ chức mã bằng các lớp (class) và đối tượng (object).

Bốn tính chất cốt lõi:

- **Đóng gói (Encapsulation)**
- **Trừu tượng (Abstraction)**
- **Kế thừa (Inheritance)**
- **Đa hình (Polymorphism)**

Mỗi phần bên dưới sẽ có giải thích ngắn và ví dụ mã riêng (code cell).

## 1) Đóng gói (Encapsulation)

**Ý tưởng:** gom dữ liệu & hành vi vào 1 lớp, che giấu trạng thái nội bộ để tránh truy cập ngoài ý muốn.
Đóng gói có 2 ý chính:
- Gom thuộc tính (dữ liệu) & phương thức (hành vi) lại với nhau trong class.
- Che giấu trạng thái nội bộ (private/protected) để tránh truy cập ngoài ý muốn.

In [ ]:

class BankAccount:
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self.__balance = float(balance)  # private (name mangling)

    def balance(self) -> float:
        """Trả về số dư (không cho set trực tiếp)"""
        return self.__balance

    def deposit(self, amount: float):
        if amount <= 0:
            raise ValueError('Deposit must be positive')
        self.__balance += amount
        return self.__balance

    def withdraw(self, amount: float):
        if amount <= 0:
            raise ValueError('Withdraw must be positive')
        if amount > self.__balance:
            raise ValueError('Insufficient funds')
        self.__balance -= amount
        return self.__balance

# Thử nghiệm
acc = BankAccount('Lan', 100)
print('Owner:', acc.owner)
print('Balance (via property):', acc.balance)
acc.deposit(50)
print('After deposit:', acc.balance)

# Ghi chú: cố tình set acc.__balance từ bên ngoài không thay đổi nội bộ do name mangling.
acc.__balance = 9999
print('After external set attempt, acc.__balance attribute exists externally, but property still returns:', acc.balance)
# Truy cập thực sự (không khuyến nghị): acc._BankAccount__balance = 9999


Owner: Lan
Balance (via property): 100.0
After deposit: 150.0
After external set attempt, acc.__balance attribute exists externally, but property still returns: 150.0


## 2) Kế thừa (Inheritance)

**Ý tưởng:** lớp con tái sử dụng/ mở rộng hành vi từ lớp cha. Dùng `super()` để gọi constructor/method của cha. Không lạm dụng kế thừa nếu chỉ cần reuse một phần hành vi (xem composition).

In [ ]:

# Ví dụ super()
class Vehicle:
    def __init__(self, make, model):
        self.make = make
        self.model = model

class Car(Vehicle):
    def __init__(self, make, model, doors):
        super().__init__(make, model)
        self.doors = doors

c = Car('Honda', 'Civic', 4)
print('Car:', c.make, c.model, c.doors)


## 3) Trừu tượng (Abstraction)

**Ý tưởng:** chỉ lộ ra giao diện cần dùng, ẩn chi tiết triển khai.

In [ ]:

from abc import ABC

class PaymentProcessor(ABC):
    """Giao diện cho bộ xử lý thanh toán"""

    def pay(self, amount: float) -> bool:
        pass

class PaypalProcessor(PaymentProcessor):
    def pay(self, amount: float) -> bool:
        print(f"Thanh toán {amount} bằng PayPal...")
        return True

class CardProcessor(PaymentProcessor):
    def pay(self, amount: float) -> bool:
        print(f"Thanh toán {amount} bằng thẻ tín dụng...")
        return True

def checkout(processor: PaymentProcessor, amount: float):
    return processor.pay(amount)

print('Pay with PayPal:', checkout(PaypalProcessor(), 50.0))
print('Pay with Card:  ', checkout(CardProcessor(), 75.0))


## 4) Đa hình (Polymorphism)

**Ý tưởng:** cùng 1 giao diện có thể có nhiều hiện thực. Thực hiện bằng overriding, duck typing hoặc operator overloading. Ví dụ gọi `speak()` trên list các `Animal` đã là đa hình.

In [ ]:

class Animal:
    def __init__(self, name: str):
        self.name = name

    def speak(self):
        raise NotImplementedError('Subclasses must implement speak')

class Dog(Animal):
    def speak(self):
        return f"{self.name} says Woof"

class Cat(Animal):
    def speak(self):
        return f"{self.name} says Meow"

animals = [Dog('Rex'), Cat('Mèo Mun')]
for a in animals:
    print(a.speak())

### Liskov Substitution Principle (LSP)

Nếu S là subtype của T, thì đối tượng kiểu T có thể thay bằng S mà không làm thay đổi tính đúng đắn chương trình. Tránh thiết kế khiến subclass phá vỡ contract của superclass. Ví dụ: Bird.fly() và Penguin không thể fly => tách interface Flyable.

## Ví dụ tích hợp (Mini-system phương tiện giao thông)
Kết hợp các tính chất OOP: abstract Vehicle, PetrolCar, ElectricCar (encapsulation cho battery).

In [ ]:

from abc import ABC, abstractmethod

class VehicleBase(ABC):
    @abstractmethod
    def start(self):
        pass

class PetrolCar(VehicleBase):
    def __init__(self, make):
        self.make = make
    def start(self):
        return f'{self.make} starts with petrol engine'

class ElectricCar(VehicleBase):
    def __init__(self, make, battery_level=100):
        self.make = make
        self._battery = battery_level  # encapsulated
    def start(self):
        if self._battery < 10:
            return f'{self.make} cannot start: battery low'
        return f'{self.make} starts silently (electric)'

fleet = [PetrolCar('Ford'), ElectricCar('Tesla', battery_level=5), ElectricCar('Nissan', battery_level=80)]
for v in fleet:
    print(v.start())


## Bài tập thực hành

1. Viết lớp `Student` (encapsulation cho grades).
2. Viết abstract class `Shape` và các lớp `Circle`, `Rectangle` (tính tổng diện tích - polymorphism).